<h2>Code to scrape data for company information & get price data from yfinnace API</h2>

In [1]:
import pandas as pd
import numpy as np

#Data Scraping Packages
import requests
from bs4 import BeautifulSoup

#historical price downloader package
import yfinance as yf  

In [2]:
#Data Scraper Code

website_text = requests.get('https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average#2010s').text
soup = BeautifulSoup(website_text,'xml')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['name', 'exchange_id', 'ticker', 'sector', 'Date Added','d'])
df.drop(['d','Date Added'], axis=1, inplace=True)
df.drop([0], axis=0, inplace=True)
df['ticker'] = df['ticker'].str.replace('NYSE:', '')
df['exchange_id'] = df['exchange_id'].str.replace('NYSE','1')
df['exchange_id'] = df['exchange_id'].str.replace('NASDAQ','2')
df['exchange_id'] = df['exchange_id'].astype(int)
df.insert(4, 'created_date', pd.datetime.now().replace(microsecond=0))
df.insert(5, 'last_updated_date', pd.datetime.now().replace(microsecond=0))

def rearrange_list(input_list, input_item_to_move, input_item_insert_here):
    '''
    Helper function to re-arrange the order of items in a list.
    Useful for moving column in pandas dataframe.

    Inputs:
        input_list - list
        input_item_to_move - item in list to move
        input_item_insert_here - item in list, insert before 

    returns:
        output_list
    '''
    # make copy for output, make sure it's a list
    output_list = list(input_list)

    # index of item to move
    idx_move = output_list.index(input_item_to_move)

    # pop off the item to move
    itm_move = output_list.pop(idx_move)

    # index of item to insert here
    idx_insert = output_list.index(input_item_insert_here)

    # insert item to move into here
    output_list.insert(idx_insert, itm_move)

    return output_list

ls_cols = df.columns
ls_cols = rearrange_list(ls_cols, 'exchange_id', 'name')
ls_cols = rearrange_list(ls_cols, 'ticker', 'name')
df=df[ls_cols]
df.to_csv(r'C:\Users\Rahul Kalubowila\Desktop\Year 3\2nd Sem\Database\Stock\symbol.csv',index=True)
df

,exchange_id,ticker,name,sector,created_date,last_updated_date
1,1,MMM,3M,Conglomerate,2020-02-03 21:21:51,2020-02-03 21:21:51
2,1,AXP,American Express,Financial services,2020-02-03 21:21:51,2020-02-03 21:21:51
3,2,AAPL,Apple Inc.,Information technology,2020-02-03 21:21:51,2020-02-03 21:21:51
4,1,BA,Boeing,Aerospace manufacturer and Arms industry,2020-02-03 21:21:51,2020-02-03 21:21:51
5,1,CAT,Caterpillar Inc.,Construction and Mining,2020-02-03 21:21:51,2020-02-03 21:21:51
6,1,CVX,Chevron Corporation,Petroleum industry,2020-02-03 21:21:51,2020-02-03 21:21:51
7,2,CSCO,Cisco Systems,Information technology,2020-02-03 21:21:51,2020-02-03 21:21:51
8,1,KO,The Coca-Cola Company,Food industry,2020-02-03 21:21:51,2020-02-03 21:21:51
9,1,DOW,Dow Inc.,Chemical industry,2020-02-03 21:21:51,2020-02-03 21:21:51
10,1,XOM,ExxonMobil,Petroleum industry,2020-02-03 21:21:51,2020-02-03 21:21:51


In [3]:
#Code to download Historical Prices using yfinance API

tickers = df['ticker']
tickers.drop(tickers.index[8], inplace=True)
tickers.reset_index(drop=True, inplace=True)
tickers.index = np.arange(1, len(tickers) + 1)

data = pd.DataFrame ()

for ticker in tickers:
# Get the data for the stock Apple by specifying the stock ticker, start date, and end date
    data = data.append(yf.download(ticker,'2019-02-28','2019-02-28'))
    
sample = data.copy()

sample['price_date'] = sample.index
sample.reset_index(drop=True, inplace=True)
sample.index = np.arange(1, len(sample) + 1)
sample['ticker'] = tickers 
sample['data_vendor_id'] = 1

sample.to_csv(r'C:\Users\Rahul Kalubowila\Desktop\Year 3\2nd Sem\Database\Stock\daily_price.csv',index=True)
sample

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,Open,High,Low,Close,Adj Close,Volume,price_date,ticker,data_vendor_id
1,209.050003,209.050003,206.960007,207.389999,202.017090,2121700,2019-02-28,MMM,1
2,107.220001,108.449997,107.080002,107.739998,106.249207,3295800,2019-02-28,AXP,1
3,174.320007,174.910004,172.919998,173.149994,171.318542,28215400,2019-02-28,AAPL,1
4,438.700012,441.420013,437.079987,439.959991,432.211761,5063100,2019-02-28,BA,1
5,138.550003,139.000000,137.139999,137.339996,133.477356,4024000,2019-02-28,CAT,1
6,120.000000,120.160004,118.489998,119.580002,116.066391,6406500,2019-02-28,CVX,1
7,51.549999,51.959999,51.349998,51.770000,50.367107,30708500,2019-02-28,CSCO,1
8,45.119999,45.610001,45.110001,45.340000,43.947472,22533900,2019-02-28,KO,1
9,79.449997,79.750000,78.709999,79.029999,76.255180,14597900,2019-02-28,XOM,1
10,198.039993,198.399994,196.009995,196.699997,193.505081,2964300,2019-02-28,GS,1


In [4]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29 entries, 1 to 29
Data columns (total 9 columns):
Open              29 non-null float64
High              29 non-null float64
Low               29 non-null float64
Close             29 non-null float64
Adj Close         29 non-null float64
Volume            29 non-null int64
price_date        29 non-null datetime64[ns]
ticker            29 non-null object
data_vendor_id    29 non-null int64
dtypes: datetime64[ns](1), float64(5), int64(2), object(1)
memory usage: 2.3+ KB
